### **Optimización de inferencia en LLM**



In [ ]:
import time
import math
import numpy as np
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Usa un modelo pequeño para las demostraciones
MODEL_NAME = "gpt2"


### **Desafíos de la inferencia**

En producción (o incluso en un laboratorio con Docker) nos preocupan:

- **Latencia promedio** (p50): tiempo medio por petición.
- **Latencia en la cola** (p95, p99): cuánto sufren los *tail users*.
- **Throughput**: peticiones/segundo que somos capaces de sostener.
- **Costo**: tiempo de CPU/GPU, energía y costo por token generado.
- **Uso de memoria**:
  - Memoria estática: pesos del modelo.
  - Memoria dinámica: activaciones, cachés, buffers internos.

En un LLM autoregresivo:

- La complejidad de atención por token es normalmente `O(L * d)` por cabecera, donde:
  - `L` = longitud de contexto (prompt + tokens ya generados).
  - `d` = dimensión del embedding / head.
- Cada token nuevo aumenta `L` y por tanto el cómputo crece con la longitud de la secuencia, a menos que reutilicemos resultados (K-V caching).


### **K-V caching**

#### **Recordatorio de la atención**

En una capa de atención, para cada token tenemos:

- `Q` (query), `K` (key), `V` (value).
- La atención se calcula como:

$$
\text{Attention}(Q, K, V) = \text{softmax}\left(\frac{Q K^\top}{\sqrt{d_k}}\right) V
$$

En un modelo autoregresivo, cuando generas el token `t`:

- `Q_t` se calcula solo para la nueva posición.
- `K_1,...,K_t` y `V_1,...,V_t` se usan para atender todo el contexto previo.

Si **no** usamos K-V caching:

- Para cada token nuevo, vuelves a calcular todas las `K` y `V` de todos los tokens anteriores.
- Esto implica recomputar trabajo que ya hiciste.

Con **K-V caching**:

- Guardas `K_1,...,K_t` y `V_1,...,V_t` en un buffer (`past_key_values`).
- Para el siguiente token solo calculas `K_{t+1}, V_{t+1}` y concatenas.
- La complejidad por token se acerca a `O(t)` en vez de `O(t^2)` en la práctica.


In [ ]:
# Demostración más controlada de K-V caching con gpt2

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
modelo = AutoModelForCausalLM.from_pretrained(MODEL_NAME).to(device)
modelo.eval()

prompt = "Explain in one sentence what key-value caching is in transformer models."
inputs = tokenizer(prompt, return_tensors="pt").to(device)

max_new_tokens = 30

# 1) Generación directa con generate (usa cache internamente)
start = time.time()
with torch.no_grad():
    out_generate = modelo.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        use_cache=True
    )
t_generate = time.time() - start
print(f"Tiempo usando generate+use_cache=True: {t_generate:.3f}s")

# 2) Bucle manual SIN usar past_key_values explícito (no guardamos cache entre pasos)
def autoregressive_no_cache(modelo, input_ids, max_steps):
    ids = input_ids
    for _ in range(max_steps):
        with torch.no_grad():
            outputs = modelo(input_ids=ids)  # recalcula todo
            next_token_logits = outputs.logits[:, -1, :]
            next_token = torch.argmax(next_token_logits, dim=-1, keepdim=True)
        ids = torch.cat([ids, next_token], dim=-1)
    return ids

start = time.time()
out_no_cache = autoregressive_no_cache(modelo, inputs["input_ids"], max_new_tokens)
t_no_cache = time.time() - start
print(f"Tiempo autoregresivo sin cache explícito: {t_no_cache:.3f}s")

print("\nSalida (corta) con generate:")
print(tokenizer.decode(out_generate[0][: len(inputs['input_ids'][0]) + 40], skip_special_tokens=True))


#### **Uso explícito de `past_key_values`**

Podemos mostrar aún más claro el mecanismo:

1. Primer *forward* sobre el *prompt* con `use_cache=True` -> obtenemos `past_key_values`.
2. Luego, en cada paso:
   - Pasamos solo el último token + `past_key_values`.
   - El modelo devuelve logits y un nuevo `past_key_values` actualizado.


In [ ]:
def autoregressive_with_kv_cache(model, input_ids, max_steps):
    # Paso 1: procesar todo el prompt con cache
    with torch.no_grad():
        outputs = model(input_ids=input_ids, use_cache=True)
    past_kv = outputs.past_key_values
    generated = input_ids

    for _ in range(max_steps):
        last_token = generated[:, -1:]
        with torch.no_grad():
            outputs = model(input_ids=last_token, past_key_values=past_kv, use_cache=True)
        next_token_logits = outputs.logits[:, -1, :]
        next_token = torch.argmax(next_token_logits, dim=-1, keepdim=True)
        past_kv = outputs.past_key_values
        generated = torch.cat([generated, next_token], dim=-1)

    return generated

start = time.time()
out_kv = autoregressive_with_kv_cache(modelo, inputs["input_ids"], max_new_tokens)
t_kv = time.time() - start
print(f"Tiempo autoregresivo con past_key_values: {t_kv:.3f}s")

print("\nSalida (corta) con KV cache explícito:")
print(tokenizer.decode(out_kv[0][: len(inputs['input_ids'][0]) + 40], skip_special_tokens=True))


### **Early Exit y Knowledge Distillation**

#### **Early Exit**

En un modelo profundo (muchas capas):

- Las primeras capas capturan patrones locales.
- Las capas intermedias capturan estructuras más abstractas.
- Las últimas capas refinan la predicción.

Idea de *early exit*:

- Añadir "clasificadores auxiliares" en capas intermedias.
- En tiempo de inferencia, en cada capa calculas:
  - Probabilidades sobre el siguiente token (o clase).
  - Una medida de confianza (por ejemplo, entropía o gap entre top-1 y top-2).
- Si la confianza supera un umbral, puedes **detenerte** sin pasar por las capas restantes.

**Ventajas**:

- Para ejemplos "fáciles", reduces latencia porque no recorres todas las capas.
- Para ejemplos difíciles, el modelo sigue usando la profundidad completa.

**Desventajas**:

- Arquitectura y código de inferencia más complejos.
- Necesitas una fase de entrenamiento adicional para esos clasificadores intermedios.

#### **Knowledge Distillation (KD) con temperatura**

En KD, el *teacher* genera logits `z_teacher` y el *student* logits `z_student`.  
Usamos una **temperatura `T`** para suavizar la distribución:

$$
p_i^{(teacher)} = \frac{\exp(z_i / T)}{\sum_j \exp(z_j / T)}, \quad
p_i^{(student)} = \frac{\exp(z_i / T)}{\sum_j \exp(z_j / T)}
$$

Con `T > 1`, la distribución es más suave y revela "conocimiento oscuro" (dark knowledge) sobre clases secundarias.

La loss típica es:

$$
L = T^2 \cdot \text{KL}(p^{(teacher)} \,\|\, p^{(student)})
$$


In [ ]:
# Ejemplo más explícito de KD con distintas temperaturas

torch.manual_seed(0)

vocab_size = 10
teacher_logits = torch.randn(vocab_size) * 3.0  # teacher más "afilado"
student_logits = torch.randn(vocab_size, requires_grad=True)

def distillation_step(teacher_logits, student_logits, temperature):
    softmax = torch.nn.Softmax(dim=-1)
    log_softmax = torch.nn.LogSoftmax(dim=-1)
    kl_div = torch.nn.KLDivLoss(reduction="batchmean")

    teacher_probs = softmax(teacher_logits / temperature).detach()
    student_log_probs = log_softmax(student_logits / temperature)

    loss = kl_div(student_log_probs, teacher_probs) * (temperature ** 2)
    return loss, teacher_probs, torch.exp(student_log_probs)

for T in [1.0, 2.0, 5.0]:
    student_logits.grad = None
    loss, p_teacher, p_student = distillation_step(teacher_logits, student_logits, T)
    loss.backward()
    print(f"Temperature={T}: Loss={loss.item():.6f}, grad_norm={student_logits.grad.norm().item():.6f}")
    print(f"  Teacher probs (top-3): {torch.topk(p_teacher, 3).values}")
    print(f"  Student probs (top-3): {torch.topk(p_student, 3).values}")
    print("-" * 60)


### **Decoding avanzado: Speculative y Parallel Decoding**

#### **Speculative Decoding (idea intuitiva)**

Imagina:

- Modelo pequeño y rápido (*draft model*).
- Modelo grande y caro (*target model*).

Pasos:

1. El modelo pequeño propone `k` tokens (ejemplo: 4) de golpe.
2. El modelo grande evalúa esos `k` tokens en bloque y calcula sus probabilidades reales.
3. Si la secuencia propuesta es lo bastante "compatible" con el modelo grande:
   - Aceptas varios tokens de una sola vez.
   - Solo "corriges" cuando haya divergencias grandes.

Resultado:

- Menos llamadas *full forward* al modelo grande.
- Ahorro de latencia, especialmente con contextos largos.

#### **Parallel/Beam Decoding**

- **Beam search** mantiene varias hipótesis en paralelo:
  - Aumenta calidad a costa de cómputo.
- **Parallel decoding** (investigación actual):
  - Intenta predecir varios tokens futuros en paralelo y luego corregir.
  - Más complicado de implementar correctamente, pero prometedor para latencias bajas.


In [ ]:
import torch, time

# Asumimos que:
# - modelo (draft_model y target_model) ya está cargado
# - tokenizer ya está cargado
# - device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def speculative_step(draft_model, target_model, tokenizer, prompt, max_new_tokens=30, draft_k=4):
    # Tokenizamos el prompt una sola vez
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    generated = inputs["input_ids"]  # [1, L_inicial]

    # Número de iteraciones: cada paso propone draft_k tokens
    num_steps = max_new_tokens // draft_k

    for _ in range(num_steps):
        # Atención coherente con la longitud actual de la secuencia
        attention_mask = torch.ones_like(generated, device=device)

        # 1) Draft model propone draft_k tokens adicionales
        with torch.no_grad():
            draft_out = draft_model.generate(
                input_ids=generated,
                attention_mask=attention_mask,
                max_new_tokens=draft_k,
                do_sample=True,
                top_k=50,
                top_p=0.95,
                pad_token_id=tokenizer.eos_token_id,  # evita el warning
            )

        # Nos quedamos solo con los últimos draft_k tokens generados
        proposed = draft_out[:, -draft_k:]  # [1, draft_k]

        # 2) Target model los "evalúa" (en esta demo, simplemente los aceptamos)
        # En una implementación real, aquí usarías target_model(**...) para
        # comparar distribuciones y aceptar/rechazar token a token.
        generated = torch.cat([generated, proposed], dim=-1)  # [1, L + draft_k]

    return generated


#  Demostración rápida
short_prompt = "List three benefits of speculative decoding:"
start = time.time()
out_spec = speculative_step(modelo, modelo, tokenizer, short_prompt, max_new_tokens=20, draft_k=4)
t_spec = time.time() - start

print(f"Tiempo (toy speculative con modelo==draft==target): {t_spec:.3f}s")
print(tokenizer.decode(out_spec[0], skip_special_tokens=True))


### **Cuantización en detalle: simétrica y asimétrica**

Queremos mapear valores reales `x` a enteros `q`.

#### **Cuantización simétrica**

- Escala `s`:
  - Sea `a = max(|x|)` y el rango entero `[-Qmax, Qmax]` (ejemplo: `Qmax = 127` para 8 bits signed).
  - Definimos `s = a / Qmax`.
- Cuantizamos:

$$
q = \text{round}(x / s)
$$

- De-cuantizamos:

$$
\hat{x} = s \cdot q
$$

#### **Cuantización asimétrica**

- Para rangos que no están centrados en 0.

Sea:

- `x_min`, `x_max`.
- `Qmin`, `Qmax` (por ejemplo, `[0, 255]` para 8 bits unsigned).

Definimos:

$$
s = \frac{x_{\max} - x_{\min}}{Q_{\max} - Q_{\min}}, \quad
z = \text{round}\left(Q_{\min} - \frac{x_{\min}}{s}\right)
$$

Cuantizamos:

$$
q = \text{round}\left(\frac{x}{s}\right) + z
$$

De-cuantizamos:

$$
\hat{x} = s \cdot (q - z)
$$


In [ ]:
# Implementación sencilla de cuantización simétrica y asimétrica para un vector

def quantize_symmetric(x, num_bits=8):
    qmax = 2 ** (num_bits - 1) - 1  # ej. 127
    a = np.max(np.abs(x))
    if a == 0:
        scale = 1.0
        q = np.zeros_like(x, dtype=np.int8)
    else:
        scale = a / qmax
        q = np.round(x / scale).astype(np.int8)
        q = np.clip(q, -qmax-1, qmax)  # aseguramos rango
    x_hat = scale * q.astype(np.float32)
    return q, x_hat, scale

def quantize_asymmetric(x, num_bits=8):
    qmin = 0
    qmax = 2 ** num_bits - 1  # ej. 255
    x_min = np.min(x)
    x_max = np.max(x)
    if x_max == x_min:
        scale = 1.0
        zero_point = 0
        q = np.zeros_like(x, dtype=np.uint8)
    else:
        scale = (x_max - x_min) / (qmax - qmin)
        zero_point = int(round(qmin - x_min / scale))
        zero_point = max(qmin, min(qmax, zero_point))
        q = np.round(x / scale + zero_point).astype(np.int32)
        q = np.clip(q, qmin, qmax).astype(np.uint8)
    x_hat = scale * (q.astype(np.float32) - zero_point)
    return q, x_hat, scale, zero_point

np.random.seed(0)
x = np.random.randn(10).astype(np.float32) * 10 + 5  # distribución no centrada

q_sym, x_sym, s_sym = quantize_symmetric(x)
q_asym, x_asym, s_asym, z_asym = quantize_asymmetric(x)

def mse(a, b):
    return float(np.mean((a - b) ** 2))

print("Vector original:", x)
print("\n Simétrica")
print("q_sym:", q_sym)
print("x_hat_sym:", x_sym)
print("scale:", s_sym)
print("MSE sym:", mse(x, x_sym))

print("\n Asimétrica ")
print("q_asym:", q_asym)
print("x_hat_asym:", x_asym)
print("scale:", s_asym, "zero_point:", z_asym)
print("MSE asym:", mse(x, x_asym))


#### **Cuantización por canal/por tensor**

En modelos grandes:

- Puedes cuantizar:
  - **Por tensor**: un `scale` y `zero_point` global por tensor de pesos.
  - **Por canal**: un `scale` por cada canal de salida (por ejemplo, por fila de una matriz de pesos de una capa lineal).
- La cuantización por canal suele preservar mejor la calidad, al precio de más metadatos (más escalas).


In [ ]:
# Ejemplo: cuantización simétrica por fila en una matriz de pesos

W = np.random.randn(4, 8).astype(np.float32) * 0.5

def quantize_per_row_symmetric(W, num_bits=8):
    rows, cols = W.shape
    qW = np.zeros_like(W, dtype=np.int8)
    scales = np.zeros(rows, dtype=np.float32)
    for i in range(rows):
        q, x_hat, s = quantize_symmetric(W[i], num_bits=num_bits)
        qW[i] = q
        scales[i] = s
    return qW, scales

def dequantize_per_row_symmetric(qW, scales):
    rows, cols = qW.shape
    W_hat = np.zeros_like(qW, dtype=np.float32)
    for i in range(rows):
        W_hat[i] = scales[i] * qW[i].astype(np.float32)
    return W_hat

qW, scales = quantize_per_row_symmetric(W)
W_hat = dequantize_per_row_symmetric(qW, scales)

print("Matriz original W:\n", W)
print("\nMatriz cuantizada qW:\n", qW)
print("\nMatriz recosntruida W_hat:\n", W_hat)
print("\nMSE por fila:", np.mean((W - W_hat) ** 2, axis=1))


### **Juntando todo: K-V caching + cuantización**

La idea general para un despliegue tipo "Llama3 8-bit + KV cache" es:

1. **Modelo cuantizado** (por ejemplo, en 8 bits):
   - Pesos almacenados en `int8` o formato comprimido.
   - Matrices de proyección (`W_q, W_k, W_v, W_o`) y MLPs en bajo bit.
2. **Inferencia autoregresiva con K-V cache**:
   - Se guarda `past_key_values` en memoria (a menudo también comprimido/optimizados).
   - Cada nuevo token reutiliza los K y V de pasos previos.
3. **Pipeline típico**:
   - Pre-procesas el *prompt*.
   - Forward del *prompt* completo para inicializar cachés.
   - Generación token a token:
     - Cálculo del siguiente token usando pesos cuantizados.
     - Reutilización de caché para atención.

En este cuaderno no vamos a cargar un Llama3 real (su tamaño es demasiado grande para un contenedor estándar),
pero sí podemos ilustrar el flujo usando modelos pequeños y, opcionalmente, cuantización dinámica en PyTorch.


In [ ]:
# Ejemplo "mixto": gpt2 + cuantización de capas lineales (CPU) + generate (que usa K-V cache internamente)
# Nota: la cuantización dinámica de PyTorch solo funciona en CPU para módulos soportados.

from torch.ao.quantization import quantize_dynamic

small_model_name = "gpt2"
tokenizer_small = AutoTokenizer.from_pretrained(small_model_name)
model_fp32 = AutoModelForCausalLM.from_pretrained(small_model_name)

def count_parameters_mb(model):
    total_bytes = sum(p.numel() * p.element_size() for p in model.parameters())
    return total_bytes / (1024 * 1024)

print(f"Modelo FP32 en CPU: ~{count_parameters_mb(model_fp32):.2f} MB")

# Cuantización dinámica de capas lineales a int8
quantized_model = quantize_dynamic(
    model_fp32,
    {torch.nn.Linear},
    dtype=torch.qint8
)

print(f"Modelo cuantizado dinámicamente (int8 para Linear): ~{count_parameters_mb(quantized_model):.2f} MB")

prompt_prod = "Explain why KV caching is important for large language models in production."
inputs_prod = tokenizer_small(prompt_prod, return_tensors="pt")

# Medimos tiempos con generate (usa K-V cache) en FP32 y modelo cuantizado
def measure_generate_time(modelo, inputs, max_new_tokens=40):
    start = time.time()
    with torch.no_grad():
        _ = modelo.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            use_cache=True
        )
    return time.time() - start

t_fp32 = measure_generate_time(model_fp32, inputs_prod)
t_q = measure_generate_time(quantized_model, inputs_prod)

print(f"Tiempo generate FP32: {t_fp32:.3f}s")
print(f"Tiempo generate cuantizado dinámico: {t_q:.3f}s")


#### **Blueprint para Llama3 8-bit + KV cache**

En un entorno real con Llama3 y GPU (y librerías como `bitsandbytes`, `transformers`, etc.), el flujo sería similar a:

```python
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "meta-llama/Meta-Llama-3-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name)

# Requiere bitsandbytes y GPU
model_8bit = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_8bit=True,      # o load_in_4bit=True
    device_map="auto"       # distribuye en GPU(s)
)

prompt = "Explain the impact of KV caching and 8-bit quantization on LLM inference latency."
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

with torch.no_grad():
    out = model_8bit.generate(
        **inputs,
        max_new_tokens=128,
        use_cache=True,      # KV cache activado
        do_sample=True
    )

print(tokenizer.decode(out[0], skip_special_tokens=True))
```

Puntos clave:

- `load_in_8bit=True` carga los pesos en 8 bits usando kernels especializados.
- `use_cache=True` activa K-V caching en el *decoder*.
- Con esto, cada paso de generación usa:
  - Matmuls más baratos (int8).
  - Menos recomputación de atención (KV cache).
- En despliegues reales, se ajustan:
  - Tamaño de batch.
  - Longitud máxima de contexto.
  - Estrategias de batching dinámico, *streaming*, etc.


### **Experimento comparativo: FP32 sin caché vs FP32 con caché vs int8 con caché**

En esta sección podemos **medir y comparar** tres modos de inferencia:

1. **FP32 + sin K-V cache**  
2. **FP32 + con K-V cache**  
3. **Int8 (cuantización dinámica) + con K-V cache**

El objetivo es observar:

- Diferencias en **tamaño de modelo (MB)**.
- Diferencias en **tiempo de generación**.
- Una métrica simple de **tokens/segundo**.

> Nota: el experimento está pensado para correr en **CPU** con un modelo pequeño (`gpt2`) usando cuantización dinámica de PyTorch (`torch.ao.quantization`).


In [ ]:
import time
import numpy as np
import pandas as pd
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from torch.ao.quantization import quantize_dynamic

torch.set_num_threads(max(1, torch.get_num_threads()))

model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)

device_cpu = torch.device("cpu")

def load_fp32_model(name: str):
    modelo = AutoModelForCausalLM.from_pretrained(name)
    modelo.to(device_cpu)
    modelo.eval()
    return modelo

def count_parameters_mb(model: torch.nn.Module) -> float:
    total_bytes = sum(p.numel() * p.element_size() for p in modelo.parameters())
    return total_bytes / (1024 * 1024)

def measure_generate_time(model, inputs, max_new_tokens=40, use_cache=True):
    start = time.time()
    with torch.no_grad():
        _ = modelo.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            use_cache=use_cache
        )
    return time.time() - start

prompt = "Explain briefly why KV caching and quantization help LLM inference."
inputs_cpu = tokenizer(prompt, return_tensors="pt").to(device_cpu)

max_new_tokens = 40

model_fp32 = load_fp32_model(model_name)
size_fp32 = count_parameters_mb(model_fp32)

t_fp32_nocache = measure_generate_time(model_fp32, inputs_cpu, max_new_tokens=max_new_tokens, use_cache=False)
t_fp32_cache   = measure_generate_time(model_fp32, inputs_cpu, max_new_tokens=max_new_tokens, use_cache=True)

model_int8 = quantize_dynamic(
    model_fp32,
    {torch.nn.Linear},
    dtype=torch.qint8
)
size_int8 = count_parameters_mb(model_int8)

t_int8_cache = measure_generate_time(model_int8, inputs_cpu, max_new_tokens=max_new_tokens, use_cache=True)

rows = [
    {
        "mode": "FP32 - sin cache",
        "dtype": "float32",
        "use_cache": False,
        "model_size_MB": size_fp32,
        "time_s": t_fp32_nocache,
        "tokens_generated": max_new_tokens,
        "tokens_per_s": max_new_tokens / t_fp32_nocache if t_fp32_nocache > 0 else np.nan,
    },
    {
        "mode": "FP32 - con cache",
        "dtype": "float32",
        "use_cache": True,
        "model_size_MB": size_fp32,
        "time_s": t_fp32_cache,
        "tokens_generated": max_new_tokens,
        "tokens_per_s": max_new_tokens / t_fp32_cache if t_fp32_cache > 0 else np.nan,
    },
    {
        "mode": "Int8 - con cache",
        "dtype": "int8 (Linear)",
        "use_cache": True,
        "model_size_MB": size_int8,
        "time_s": t_int8_cache,
        "tokens_generated": max_new_tokens,
        "tokens_per_s": max_new_tokens / t_int8_cache if t_int8_cache > 0 else np.nan,
    },
]

df = pd.DataFrame(rows)
df = df[["mode", "dtype", "use_cache", "model_size_MB", "time_s", "tokens_generated", "tokens_per_s"]]

print("Resumen comparativo de modos de inferencia:\n")
print(df.to_string(index=False, float_format=lambda x: f"{x:8.3f}"))


#### **Preguntas y ejercicios**

Algunas preguntas a responder a partir de esta tabla:

1. ¿Cuánta diferencia hay en **model_size_MB** entre FP32 e int8?
2. ¿Se observa una mejora clara en **time_s** entre:
   - FP32 sin cache vs FP32 con cache?
   - FP32 con cache vs int8 con cache?
3. ¿Cuál de los tres modos tiene más **tokens_per_s**?
4. ¿En qué escenarios reales tendría sentido:
   - Aceptar una posible pequeña degradación de calidad por cuantización a cambio de esta mejora?
   - Combinar además técnicas como *speculative decoding* o *batching*?

5. Documenta: 
- Setup del experimento (CPU, número de threads, modelo, prompt).
- Tabla de resultados.
- Conclusiones y trade-offs.


In [ ]:
## Tus respuestas